In [ ]:
import numpy as np
import random
import pandas as pd
import copy

from collections import Counter, defaultdict
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


# World Sense Disambiguation (WSD)

Ce notebook permet d'effectuer une comparaison des performances de deux algorithmes, un d'apprentissage supervisé et un d'apprentissage semi-supervisé, pour la tâche de Word Sense Disambiguation (WSD). 

Les deux méthodes sont les suivants :
- pour la classification supervisée, utilisation d'un MLP
- pour la classification semi-supervisée, utilisation d'un algorithme de constrained K-Means

Plusieurs tests sont effectués en considérant plusieurs mots à désambiguiser pour lesquels les performances de ces deux méthodes sont évaluées. L'objectif final est de comparer ces performances pour déterminer à partir de combien de données annotées la première méthode est préférable à la deuxième et vice versa.



## Extraction des données

La classe Extractor permet d'accéder aux données et de les stocker dans des dictionnaires puis des embeddings. Elle permet de figer les données sur lesquelles les tests de classification seront ensuite faits. Elle comporte les méthodes suivantes :

- `extract_examples()` : pour extraire les données d'entraînement et de test provenant de fichiers XML FSE

- `extract_embeddings()` : pour extraire les embeddings à partir d'un fichier créé en amont, qui ne regroupe que les embeddings nécéssaires à notre jeu de données. Cette méthode est surtout utile pour drastiquement réduire le temps de chargement des embeddings

- `look_up()` : pour effectuer l'étape de look-up avant la classifcation

- `select_examples()` : pour sélectionner des données représentatives lorsqu'on ne considère pas toutes les données annotées. Pour qu'un ensemble de données soit représentatif, il faut que chaque étiquette présente dans les données totales soit présente au moins une fois dans le set d'entraînement sélectionné

- `define_instance()` : pour choisir les données qui seront classifiées, c'est-à-dire quelle proportion des données seront utilisées (80%, 50%, 25%...)


In [ ]:
class Extractor :
    
    def __init__(self, data_path, gold_path, embeddings_path, context_size):
        '''
        A partir des données, extrait les examples qui seront utilisés lors de l'apprentissage.

        INPUT
        data_path (str) : chemin pour accéder aux données d'entraînement, au format XML
        gold_path (str) : chemin pour accéder aux numéros de sens correspondant aux données
                          d'entraînement
        embeddings_path (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données
        context_size (int) : taille de la fenêtre du contexte du mot à désambiguiser

        ARGUMENTS DE CLASSE
        self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
                                 utilisation dans une phrase du corpus. 
                                 examples (list) : liste de tous les couples 
                                  ([mots du contexte], numéro de sens) pour un lemme donné
        self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        self.w2emb (dict) : associe chaque mot à son embedding
        self.selected_examples (liste) : liste d'examples qui contient au moins un example de 
                                         chaque sens du lemme choisi
        self.selected_examples_embs (liste) : même liste d'examples, mais avec les contextes 
                                              sous forme d'embedding
        self.X_train (list) : corpus d'apprentissage
        self.X_test (list) : numéros des sens correspondant au corpus d'apprentissage
        self.y_train (list) : corpus de test
        self.y_test (list) : numéros des sens correspondant au corpus d'apprentissage
        self.annotated_examples (list) : liste des données considérées comme annotées 
        self.unannotated_examples (list) : liste des données considérées comme non annotées 
        '''
        
        #récupération des données XML d'apprentissage fournies en argument
        tree = ET.parse(data_path)
        data_file = tree.getroot()

        #récupération des données txt correspondant aux gold class
        gold_file = open(gold_path, "r",encoding="utf-8")
        
        # self.w2examples (dict) : associe chaque lemme à une liste d'examples correspondant à son 
        #                          utilisation dans une phrase du corpus.
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        #
        # self.w2senses (dict) : associe chaque lemme à son ensemble de sens possible
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        self.w2examples, self.w2senses = self.extract_examples_and_senses(data_file, gold_file, context_size)

        # self.w2emb (dict) : associe chaque mot à l'embedding représentant son contexte sommé
        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding
        self.w2emb = self.extract_embeddings(embeddings_path)

        # liste d'examples qui contient au moins un example de chaque sens du lemme choisi
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples = []
        # même liste d'examples, mais avec les contextes sous forme d'embedding
        # cette liste sera définie lorsque l'instance du mot à désambiguiser sera choisi
        self.selected_examples_embs = []
        # liste des examples non selectionnés, dont les contextes sont sous forme d'embedding
        self.unannotated_set = []

        # les corpus seront définis lorsque l'instance du mot à désambiguiser sera choisi
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []

        # liste des données considérées comme annotées 
        self.annotated_examples = []
        # liste des données considérées comme non annotées 
        self.unannotated_examples = []
    
    def extract_examples_and_senses(self, data_file, gold_file, context_size):
        '''
        Extrait les données à partir des fichiers de corpus d'apprentissage et de gold classes.

        INPUT
        data_file (Element): représentation des phrases du corpus d'apprentissage
        gold_file (TextIOWrapper): fichier contenant les numéros de sens (gold class) pour chaque
                                   mot à désambiguiser
        '''
    
        # clé (str) : lemme du mot à désambiguiser
        # valeur (list) : liste de tous les couples ([mots du contexte], numéro de sens) pour ce 
        #                 lemme
        w2examples = {}
        # clé (str) : lemme du mot à désambiguiser
        # valeur (set) : ensemble des sens possible pour ce lemme
        w2senses = defaultdict(set)

        # lecture du fichier gold
        gold_file = gold_file.readlines()
        
        # index de parcours dans le fichier gold
        i_gold = 0

        # pour chaque phrase du corpus
        for text_id in data_file:
            for sentence in text_id:

                # recherche de(s) l'indice(s) de(s) l'instance(s) pour savoir où se trouve la fenêtre
                i_instances = []
                for j in range(len(sentence)):
                    if sentence[j].tag == "instance":
                        i_instances.append(j)
                
                # tant qu'il y a des instances à repérer dans la phrase
                while len(i_instances) > 0:

                    # enregistrement du lemme de l'instance
                    instance = sentence[i_instances[0]].attrib["lemma"].lower()
                    
                    # si l'instance n'a pas encore de contexte
                    if instance not in w2examples : 
                        # le crée
                        w2examples[instance] = []

                    context = []
                    # pour chaque mot de la fenêtre
                    for k in range(-context_size, context_size + 1):
                        # si le mot est l'instance, enregistrement de son lemme
                        if k == 0:
                            context.append(instance)
                        # sinon enregistrement du mot du contexte
                        elif len(sentence) > i_instances[0] + k and i_instances[0] + k >= 0:
                            context.append(sentence[i_instances[0] + k].text.lower())
                    
                    # récupération des différents sens possibles dans le fichier gold 
                    gold_class = gold_file[i_gold].split("__")[1].split("_")[1]

                    # le fichiers gold et data ayant les données dans le même ordre, les instances
                    # et les sens peuvent être enregistrés en même temps dans leur dictionnaire
                    # respectif
                    w2senses[instance].add(gold_class)
                    w2examples[instance].append((context, gold_class))
                    
                    # l'instance lue est enlevée de la to-do list et l'index de parcours du fichier 
                    # gold est incrémenté pour passer à l'instance suivante
                    i_instances.pop(0)
                    i_gold += 1

        return w2examples, w2senses
    
    def extract_embeddings(self,path_embeddings) :
        '''
        Récupère les embeddings dans le fichier généré et associe à chaque mot son
        embedding.

        INPUT
        path_embeddings (str) : chemin pour accéder aux embeddings, déjà extraits et 
                                présélectionnés pour ces données

        OUTPUT
        w2emb (dict) : associe chaque mot à son embedding
        '''

        # lecture du fichier
        file_embeddings = open(path_embeddings , "r", encoding="UTF-8")

        # clé (str) : mot à désambiguiser
        # valeur (list) : embedding du mot
        w2emb = {}

        # pour chaque mot du fichier
        for line in file_embeddings.readlines():

            # séparation du mot et de l'embedding
            splitted_line = line.split(" ")
            word = splitted_line[0]
            embedding = list(map(float,splitted_line[1:]))
            # insertion du mot et de son embedding dans le dictionnaire
            w2emb[word] = embedding

        return w2emb

    def look_up(self, context) :
        '''
        Remplace dans les mots du vecteur de contexte par leur embedding et en fait la
        somme.

        INPUT
        context (list) : liste de taille (size_window*2)+1

        OUTPUT
        context_emb (list) : liste de taille size_embedding (300 ici)
        '''

        # taille d'un embedding : 300
        emb_size = len(list(self.w2emb.values())[0]) 
        # initialisation du contexte sous forme d'embedding
        context_emb = np.zeros(emb_size)

        # pour chaque mot du contexte
        for word in context :
            # s'il est présent dans le dictionnaire d'embeddings
            if word in self.w2emb :
                # il est ajouté à l'embedding représentant le contexte
                context_emb = np.add(context_emb, np.array(self.w2emb[word])) 

        return context_emb
    
    def select_examples(self, examples, senses, sample_size):
        '''
        Choisit des examples représentatifs du corpus selon un nombre imposé de 
        données à choisir.

        INPUT
        examples (list) : liste d'examples pour le mot à désambiguiser
        n_senses (int) : nombre de sens associés à ce mot
        sample_size (float) : quantité des données considérés

        OUTPUT
        selected_examples (list) : liste d'examples qui contient au moins un example 
                                   de chaque sens
        '''

        selected_examples = []
        
        # pour chaque sens, on ajoute un example associé à ce sens, au hasard
        for sense in senses :
            selected_examples.append(random.choice(list(filter((lambda example:example[1]==sense),
                                                               examples))))
        
        # ajoute le nombre de données manquantes (non-présentes dans la liste) selectionnées au hasard
        random.shuffle(examples)
        for example in examples:
            if example not in selected_examples and len(selected_examples) < sample_size*len(examples):
                selected_examples.append(example)

        return selected_examples
    
    def select_train_test(self, x, y, senses, data_size, train_size = 0.8):
        '''
        Sépare en un train set et test set les examples représentatifs du corpus en mettant 
        au moins une étiquette de chaque sens dans le train set (et le test set si possible)

        INPUT
        x (list) : liste d'examples pour le mot à désambiguiser
        y (list) : liste d'étiquette pour chaque example de x
        senses (list) : numéros des sens associés à ce mot
        train_size (float) : quantité des données d'entraînement considérés
        '''

        # SELECTION DES DONNEES REPRESENTATIVES DU TRAIN

        # sélection au hasard d'un contexte et de son étiquette pour chaque sens
        selected_train = [random.choice(list(filter((lambda example: example[1] == sense),
                                                    zip(x,y)))) for sense in senses]
            
        # SELECTION DES DONNEES REPRESENTATIVES DU TEST

        # fait de même pour le corpus de test, lorsque possible, en ne considérant pas
        # ceux déjà sélectionnés pour le train
        selected_test = []
        for sense in senses:
            added = False
            for example in zip(x,y):
                for train_ex in selected_train:
                    # vérifie que l'example n'a pas déjà été sélectionné dans le train et
                    # le test pour ne pas faire de doublon
                    if example[1] == sense and example[1] == train_ex[1] and not(np.array_equal(example[0], train_ex[0])):
                        selected_test.append(example)
                        added = True
                        break
                if added == True:
                    break

        # SELECTION DES DONNEES RESTANTES DU TRAIN ET DU TEST

        # effectue un split sur le reste d'examples disponibles
        remaining_X = []
        remaining_y = []
        selected_train_copy = copy.deepcopy(selected_train)
        selected_train_copy.extend(selected_test)
        for contexte, sens in zip(x,y):
            to_add = True
            for ex in selected_train_copy:
                # vérifie que l'example n'a pas déjà été sélectionné dans le train et le
                # test pour ne pas faire de doublon
                if np.array_equal(contexte, ex[0]) and sens == ex[1]:
                    to_add = False
                    break
            if to_add:
                remaining_X.append(contexte)
                remaining_y.append(sens)

        try:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(remaining_X, remaining_y,train_size = train_size)
    
            # ajout des données représentatives dans les sets correspondant
            selected_X_train, selected_y_train = zip(*selected_train)
            selected_X_test, selected_y_test = zip(*selected_test)
            
            self.X_train.extend(selected_X_train)
            self.X_test.extend(selected_X_test)
            self.y_train.extend(selected_y_train)
            self.y_test.extend(selected_y_test)
            
        except:
            print(f"La taille des données ({data_size}) est trop faible. L'opération train_test_split() est impossible.")                                                                     
            self.X_train, self.X_test, self.y_train, self.y_test = set(), set(), set(), set()
                     
    def define_instance(self, instance, data_size, display=True):
        '''
        Permet de définir l'instance du mot à désambiguiser et les examples à utiliser. 
        Fixe les derniers arguments de classe selon l'instance choisie.
        Contraint le choix de classification selon le nombre de données considéré.

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float): quantité de données à considérer
        display (bool, optional): affiche une trace des options choisies
        '''

        # SET DE DONNEES ANNOTEES

        # choisit des examples d'entraînement représentatifs du corpus selon le nombre
        # imposé de données
        self.annotated_examples = self.select_examples(self.w2examples[instance],
                                                       self.w2senses[instance],
                                                       data_size)
        # liste de tous les contextes sous forme d'embedding
        X_annotated = [self.look_up(context) for context, gold in self.annotated_examples]
        # liste de tous les numéros de sens
        y_annotated = [gold for context, gold in self.annotated_examples]
        # le set des données annotées est une liste des examples représentatifs sous forme d'embeddings
        self.selected_examples_embs = list(zip(X_annotated,y_annotated))

        # SET DE DONNEES NON ANNOTEES

        # garde en mémoire les examples considérés comme non-annotés
        self.unannotated_examples = [example for example in self.w2examples[instance] 
                                     if example not in self.annotated_examples]        
        # liste de tous les contextes sous forme d'embedding
        X_unannotated = [self.look_up(context) for context,gold in self.unannotated_examples]
        # liste de tous les numéros de sens
        y_unannotated = [gold for context,gold in self.unannotated_examples]
        # le set des données non-annotées est une liste des examples sous forme d'embeddings
        self.unannotated_set = list(zip(X_unannotated,y_unannotated))

        # SET DE DONNEES A CLASSIFIER PAR MLPCLASSIFIER_WSD

        # création des corpus d'entrainement et de test
        self.select_train_test(X_annotated, y_annotated, self.w2senses[instance],data_size)
        
        # affichage des informations relatives à l'instance choisie pour MLPClassifier_WSD
        if display :
            self.classifier_display_at_instance_time(instance, data_size)
            
    
    def classifier_display_at_instance_time(self, instance, data_size):
        '''
        Affiche les informations relatives à l'instance choisie pour MLPClassifier_WSD

        INPUT
        instance (str) : mot à désambiguiser
        data_size (float) : quantité de données à considérer
        '''
        
        print("--> Instance :", instance)
        print(f'{data_size*100}% des données annotées considérées')
        print("Nombre de données d'entraînement : ", len(self.X_train))
        print("Etiquettes possibles pour cette instance : ", self.w2senses[instance])
        if self.y_train:
            print("Etiquettes présentes dans les données d'entraînement :", Counter(self.y_train))
            most_frequent_sense = max(self.y_train,key=self.y_train.count)
            occurrence_of_most_frequent_sense = self.y_train.count(most_frequent_sense)/len(self.y_train)
            #print(f"Les occurrences du sens {most_frequent_sense} sont les plus fréquentes, avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
    

    def kmeans_display_at_instance_time(self, instance, num_correct, num_examples, size):
        ''' 
        Affiche des informations relatives à l'instance choisie pour K_Means

        INPUT
        instance (str) : mot à désambiguiser
        num_correct (int) : nombre de prédictions correctes
        num_examples (int) : nombre d'examples total
        size (float) : quantité de données à considérer 
        '''
        y = [gold for tensor,gold in self.selected_examples_embs] + [gold for tensor,gold in self.unannotated_set] 
        print("--> Instance :",instance)
        print(f"Proportion de données annotées considérées sur le corpus: {size*100} %")
        print(f"Répartition des sens: {Counter(y)}")
        most_frequent_sense = max(y,key=y.count)
        occurrence_of_most_frequent_sense = y.count(most_frequent_sense)/len(y)
        print(f"Le sens le plus fréquent pour '{instance}' est le sens {most_frequent_sense} avec une proportion de {round(occurrence_of_most_frequent_sense*100,2)} %")
        #print(f"Pour '{instance}', l'accuracy est de {round(num_correct/num_examples,2)} %")
        print()

    def get_sets(self):
        return self.X_train, self.X_test, self.y_train, self.y_test
    
    def get_annotated_and_unannotated_sets(self):
        return self.selected_examples_embs, self.unannotated_set
        
    def get_annotated_examples(self):
        return self.annotated_examples
    
    def get_unannotated_examples(self):
        return self.unannotated_examples
    
    def get_embs(self):
        return self.selected_examples_embs

In [ ]:
# chemin du corpus annoté
data_path = "../donnees/FSE-1.1-191210/FSE-1.1.data.xml"
# chemin pour récupérer les gold class du corpus annoté
gold_path = "../donnees/FSE-1.1-191210/FSE-1.1.gold.key.txt"
# choix de la fenêtre du contexte
context_size = 4
# chemin pour récupérer les embeddings afin de faire l'opération look-up
# Les embeddings sont extraits de fasttext
embeddings_path = "../donnees/embeddings.txt"
# choix de l'instance à tester pour la classification
instance = "aboutir"

In [ ]:
# exemple d'utilisation d'Extractor avec le verbe "aboutir" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance(instance, 0.1, display=True)
print("\nexemple d'example avec contexte et gold class :",ext.get_annotated_examples()[0])
print("\nau format embedding : ",ext.get_embs()[0])


## Classification supervisée

L'utilisation d'un Classifieur MLP permet de prédire le sens des verbes de façon supervisée. Il s'agit de la première méthode proposée pour la tâche de WSD.

Les données utilisées sont les données annotées extraites du fichier FSE grâce à l'Extractor pour un lemme donné. Un MLPClassifier_WSD doit donc être créé pour chaque verbe dont on veut la prédiction.

MLPClassifier_WSD comporte les méthodes suivantes :
- `classify()` : pour prédire le sens du lemme associé au classifieur et calculer l'accuracy de cette prédiction


In [ ]:
class MLPClassifier_WSD :
    
    def __init__(self, X_train, X_test, y_train, y_test) :
        '''
        A partir des données, prédit le sens d'un lemme et évalue la prédiction

        ARGUMENTS DE CLASSE
        self.X_train (list) : liste des contextes d'entraînement
        self.y_train (list) : liste des gold class d'entraînement
        self.X_test (list) : liste des contextes de test
        self.y_test (list) : liste des gold class de test
        '''
        # ensembles d'entraînement et de test provenant de l'Extractor pour un lemme donné
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test 

        # prédiction du modèle
        self.y_pred = []
        # accuracy de la prédiction
        self.accuracy = 0
        
    def classify(self, display=False) :
        ''' 
        Permet de prédire le sens du verbe.

        INPUT
        display (bool, optional) : affiche une trace des options choisies
        '''
        try:
            clf = MLPClassifier(random_state=1, hidden_layer_sizes=(100,)) 
            # entraînement
            clf.fit(self.X_train, self.y_train)
            # prédiction
            self.y_pred = clf.predict(self.X_test)

            # calcul de l'accuracy
            self.accuracy = accuracy_score(self.y_pred, self.y_test)
            
            # si une trace est souhaitée
            if display :
                print("prediction :", self.y_pred)
                print("gold :", self.y_test)
                print("accuracy score : ", round(self.accuracy,2),"\n")
        except:
            print("ATTENTION: La quantité de données fournies est insuffisante: formation du test set impossible")
        

In [ ]:
# exemple d'utilisation de MLPClassifieur_WSD avec le verbe "demeurer" et 100% des données utilisées
ext = Extractor(data_path, gold_path, embeddings_path, context_size)
ext.define_instance("demeurer", 0.1, display=True)
X_train, X_test, y_train, y_test = ext.get_sets()

Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
Clf.classify(display=True)

Pour pouvoir comparer les performances de ce Classifieur MLP avec celles de l'apprentissage semi-supervisé, nous devons créer un classifieur par verbe à désambiguiser et choisir un pas de descente pour la quantité de données annotées considérées.

Pour chaque classifieur et chaque quantitée de données considérées, on effectue *n_repeat* classifications pour obtenir une accuracy moyenne représentative du classifieur. 

Par conséquent, pour :
- n_repeat = 5
- step = 0.25

nous obtiendrons pour chaque classifieur une liste d'accuracies correspondant à :
- la moyenne des accuracies de 5 prédictions 
- pour 100%, 75%, 50% et 25% des données.

In [ ]:
def get_mean_accuracy(instance, data_size, n_repeat, affichage=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné.

    INPUT
    n_repeat (int): nombre de classifications à effectuer
    display (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean (float): moyenne des accuracies calculées
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
        
    for _ in range(n_repeat) :
            
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données
        ext.define_instance(instance, data_size, False)
        X_train, X_test, y_train, y_test = ext.get_sets()
        
        if y_train :
            Clf = MLPClassifier_WSD(X_train, X_test, y_train, y_test)
            Clf.classify()
                
            accuracies.append(Clf.accuracy)
            mean = sum(accuracies)/len(accuracies)
            
        else :
            accuracies.append("impossible")
            mean = "impossible"
            
    # si la trace est souhaitée
    if display :
        print(f'{data_size*100}% des données annotées considérées')
        print("instance :", instance)
        print("accuracies :", accuracies)
        print("accuracy moyenne :", mean, "\n")

    return mean

def get_accuracies(instances, step, n_repeat, display=True):
    '''
    - Permet d'obtenir une accuracy moyenne par lemme pour une certaine quantité 
      de données considérée.
    - Produit un fichier csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    instance2acc = {instance : [] for instance in instances}
    moyennes = []
    data_sizes =[]
            
    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (100%, 75%, 50%, 25%)
        data_size = 1.0 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de son accuracy moyenne
            instance2acc[instance].append(get_mean_accuracy(instance, data_size, n_repeat, display))

        # on boucle sur les valeurs obtenues pour obtenir leur moyenne
        accuracies = [accuracies[i] for accuracies in instance2acc.values() if accuracies[i] != "impossible"]
        moyennes.append(sum(accuracies)/len(accuracies))
  
    # ajout d'une clé moyennes pour avoir les moyennes de toutes les accuracy moyennes pour chaque instance
    # pour chaque pas
    instance2acc["moyennes"] = moyennes
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    instance2acc["data_sizes"] = data_sizes 

    # export en csv des résultats
    df = pd.DataFrame(instance2acc)
    df.set_index("data_sizes")
    df.to_csv(f"accuracies_par_lemme_{n_repeat}_repet.csv")
    
    return instance2acc     

In [ ]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5

In [ ]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())
# calcul des accuracies
get_accuracies(instances, step, n_repeat)

## Classification semi-supervisée

In [ ]:
class Constrained_K_Means():
    ''' 
    Classifieur K-means pour un mot particulier
    '''

    def __init__(self, annotated_examples, not_annotated_examples):
        '''
        Instancie les différentes variables utiles pour l'algorithme du K-means

        INPUT
        annotated_examples (list) : liste des exemples qui seront considérés
                                     comme annotés pour la construction des premiers
                                     centroïdes
        non_annotated_examples (list) : liste des exemples à classifier 

        ARGUMENTS DE CLASSE
        self.annotated_examples
        self.not_annotated_examples
        self.k (int) : nombre de clusters pour ce lemme
        self.centroids (list) : liste des centroïdes sous forme d'embeddings
        self.cluster2sense (list) : associe chaque cluster (indice de la liste)
                                    à un sens
        '''

        self.annotated_examples = annotated_examples

        # les gold class ne seront utilisées que pour calculer l'accuracy
        self.not_annotated_examples = not_annotated_examples

        # initialisation de clusters : tous les examples non annotés sont associés 
        # au cluster 0
        self.clusters = np.zeros(len(not_annotated_examples))

        # détermine le nombre de sens possibles k (donc le nombre de clusters) 
        # à l'aide des données annotées, qui représentent tous les sens possibles
        self.k = len(set([example[1] for example in self.annotated_examples]))

        # initialisation des centroïdes : pour chaque sens, le centroïde 
        # correspond à la moyenne des embeddings des exemples annotés
        # Ainsi, chaque centroïde représente un sens
        self.centroids, self.cluster2sense = self.make_centroids(self.annotated_examples)


    def make_centroids(self, data):
        '''
        Calcule les coordonnées des centroïdes

        OUTPUT
        centroids (list) : liste des coordonnées des centroïdes dans l'ordre
                           des clusters
        cluster2sense (list) : à chaque indice correpond au numéro d'un sens
        '''

        centroids = []
        # clé : numéro du cluster (ie indice de la liste)
        # valeur : numéro du sens associé
        cluster2sense = []
        # regroupe les sens présents dans le corpus
        senses = set([example[1] for example in data])
        # pour chaque sens
        for sense in senses:
            # on récupère les examples correspondant à ce sens
            examples_sense = [example[0] for example in data if example[1] == sense]
            # le centroide correspond à la moyenne de tous ces examples
            centroids.append(np.mean(examples_sense, axis=0))
            # ajoute le numéro du sens
            cluster2sense.append(sense)

        return centroids, cluster2sense


    def learn_clusters(self):
        '''
        Algorithme d'apprentissage de K-Means : met à jour les coordonnées
        des centroïdes après chaque annotation d'exemple
        '''

        # CALCUL DES DISTANCES ENTRE CHAQUE EXAMPLE ET CHAQUE CENTROIDE
         
        # pour chaque couple (indice, coordonnées) dans les examples
        for i, example in enumerate(self.not_annotated_examples):
            # initialisation de la distance minimum à l'infini
            min_dist = float('inf')
            # pour chaque couple (indice, coordonnées) dans les centroides
            for j, centroid in enumerate(self.centroids):
                # calcul de la distance entre cet example et ce centroide
                d = np.sqrt(np.sum(np.square(centroid - example[0])))
                # si une distance plus faible est trouvée
                if min_dist > d:
                    # la distance ainsi que le centroide sont stockés
                    min_dist = d
                    self.clusters[i] = j
                
            # CALCUL ET MISE A JOUR DES NOUVEAUX CENTROIDES
            
            # donne l'étiquette prédite à cet exemple
            predicted_example = (self.not_annotated_examples[i][0], self.cluster2sense[int(self.clusters[i])])
            # fusionne les données annotées avec cette donnée prédite
            data = self.annotated_examples
            data.append(predicted_example)
            # pour mettre à jour les centroides
            self.make_centroids(data)

    def accuracy(self): 
        '''
        OUTPUT
        - correct (int) : nombre de fois où le sens prédit est le bon
        - len(self.not_annotated_examples) (int): nombre total d'examples non annotés
        - predicted_sense2count (dict): dictionnaire qui associe à chaque sens le nombre de fois où il a été prédit
        '''

        correct = 0
        correctly_predicted_sense2count = {}
        for i in range(len(self.not_annotated_examples)):
            expected_sense = self.not_annotated_examples[i][1]
            predicted_sense = self.cluster2sense[int(self.clusters[i])]
            # on compte le nombre de fois où le sens prédit est le bon (sera utilisé pour calculer la micro-average)
            if expected_sense == predicted_sense:
                correct += 1
                # on compte le nombre de fois où chaque sens est correctement prédit (sera utilisé pour calculer la macro-average)
                correctly_predicted_sense2count[predicted_sense] = correctly_predicted_sense2count.get(predicted_sense, 0) + 1
        return correct, len(self.not_annotated_examples), correctly_predicted_sense2count

In [ ]:
def get_mean_macro_and_micro_averages(instance, data_size, n_repeat, display=True):
    '''
    Retourne une moyenne d'accuracies pour un nombre de classifications donné.

    INPUT
    instance (str): nom de l'instance à utiliser
    data_size (float): proportion de données à utiliser
    n_repeat (int): nombre de classifications à effectuer
    display (bool, optional): affiche une trace des options choisies

    OUTPUT
    mean (float): moyenne des accuracies calculées
    '''

    # liste de l'accuracy à chaque répétition
    accuracies = []
    macro_average = []
    n_repeat_micro_averages = []
    n_repeat_macro_averages = []
        
    for _ in range(n_repeat) :

        # ETAPE 1 ################### Clustering #########################
        # pour chaque itération, un nouveau classifieur est créé, qui s'appuie toujours sur 
        # la même quantité de données mais avec un autre tirage des données
        # par exemple, pour 75% des données, des accuracies différentes seront obtenues en
        # fonction des tirages de ces données

        ext.define_instance(instance, data_size, False)
        
        annotated_set, unannotated_set = ext.get_annotated_and_unannotated_sets()
        annotated_set = [(np.array(embedding),sense) for embedding,sense in annotated_set]
        unannotated_set = [(np.array(embedding),gold) for embedding,gold in unannotated_set]
        k_Means = Constrained_K_Means(annotated_set, unannotated_set)
        k_Means.learn_clusters()


        # ETAPE 2 ################### Evaluation #########################
        correct, num_examples, correctly_predicted_sense2count = k_Means.accuracy()
        
        if _ == n_repeat-1:
            ext.kmeans_display_at_instance_time(instance, correct, num_examples, data_size)
        
        # préparation du calcul de la micro-average moyenne: 
        # pour chaque instance, on calcule l'accuracy (nb correct/ nb total d'examples), mais puisqu'on
        # fait cette prédictions n_repeat fois, on stocke l'accuracy dans une liste pour en faire la moyenne
        n_repeat_micro_averages.append(correct/num_examples)
        # préparation du calcul de la macro-average moyenne:
        # pour chaque instance, on calcule quelle a été la précision par sens. Pour cela,
        # on stocke les sens attendus dans le dictionnaire expected_sense2count et on compare avec
        # les sens prédits dans le dictionnaire correctly_predicted_sense2count
        expected_senses = [gold for tensor,gold in annotated_set] + [gold for tensor,gold in unannotated_set] 
        expected_sense2count = Counter(expected_senses)
        for sense in correctly_predicted_sense2count.keys():
            # on stocke les précisions par sens dans une liste
            macro_average.append(correctly_predicted_sense2count[sense]/expected_sense2count[sense])
        n_repeat_macro_averages.append(sum(macro_average)/len(macro_average))

    
    mean_macro_average_score = sum(n_repeat_macro_averages)/len(n_repeat_macro_averages)
    print(f"Macro-average : {round(mean_macro_average_score,2)}") #une accuracy par sens et tu divises par le nb de sens
    mean_micro_averages = sum(n_repeat_micro_averages)/len(n_repeat_micro_averages)
    print(f"Micro-average : {round(mean_micro_averages,2)}")
    print("\n--------------------------------------------------\n")
    return mean_macro_average_score, mean_micro_averages


def get_accuracies(instances, step, n_repeat, display=True):
    '''
    - Permet d'obtenir une accuracy moyenne par lemme pour une certaine quantité 
      de données considérée.
    - Produit un fichier csv.

    INPUT
    instances (list): liste des mots à désambiguiser
    step (float): pas de descente dans la quantité de données à considérer
    n_repeat (int): nombre de classifications tests à effectuer

    OUTPUT
    instance2acc (dict) : associe à chaque instance sa liste d'accuracies moyennes
    '''

    # clé : mot à désambiguiser
    # valeur : liste des accuracies moyennes à chaque pas de descente
    averages2score = {"macro_average":[], "micro_average":[]}
    mean_macro_averages_for_kmeans = []
    mean_micro_averages_for_kmeans = []
    data_sizes =[]
    

    # pour chaque quantité de données observée
    for i in range(round(1.0/step)): 
                
        # calcul de la taille des données avec le pas de descente (75%, 50%, 25%, 1%)
        data_size = 0.75 - (step*float(i))
        data_sizes.append(data_size)
        
        # pour chaque mot à désambiguiser
        for instance in instances :
            # calcul de son macro_average et micro_average pour n_repeat classifications
            macro_average, micro_average = get_mean_macro_and_micro_averages(instance, data_size, n_repeat, display)
            averages2score["macro_average"].append(macro_average)
            averages2score["micro_average"].append(micro_average)
        # on boucle sur les valeurs obtenues pour obtenir leur moyenne
        mean_macro_averages_for_kmeans.append(sum([accuracies[i] for accuracies in averages2score["macro_average"]])/len(averages2score["macro_average"]))
        mean_micro_averages_for_kmeans.append(sum([accuracies[i] for accuracies in averages2score["micro_average"]])/len(averages2score["micro_average"]))
  
    # ajout d'une clé macro_average_moyenne_du_kmeans pour avoir les moyennes de toutes les accuracy moyennes pour chaque instance
    # pour chaque pas
    averages2score["macro_average_moyenne_du_kmeans"] = mean_macro_averages_for_kmeans
            
    # ajout d'une clé pour stocker les tailles des données à chaque pas
    averages2score["micro_average_moyenne_du_kmeans"] = mean_micro_averages_for_kmeans 

    # export en csv des résultats
    df = pd.DataFrame(averages2score)
    df.set_index("data_sizes")
    df.to_csv(f"kmeans_accuracies_per_lemma_{n_repeat}_repet.csv")
    
    return averages2score



In [ ]:
# pas de descente dans la quantité de données considérées
step = 0.25
# nombre de classifications pour un classifieur pour obtenir une accuracy moyenne
n_repeat = 5
# nombre d'itérations maximal
max_iter = 10

In [ ]:
# récupération de tous les verbes du corpus à désambiguiser
instances = list(ext.w2examples.keys())[:3]
# calcul des accuracies
get_accuracies(instances, step, n_repeat)

In [ ]:
# merge csv classifier et csv kmeans pour avoir les deux dans le même csv en alterant les colonnes de chacun des csv
# (so not this)
def merge_csv(classifier_csv, kmeans_csv):
    '''
    - Permet de fusionner les deux csv obtenus pour les deux méthodes de classification
      dans un seul csv.

    INPUT
    classifier_csv (str): nom du csv obtenu pour la méthode de classification
    kmeans_csv (str): nom du csv obtenu pour la méthode de kmeans

    OUTPUT
    merged_csv (str): nom du csv fusionné
    '''
    classifier_df = pd.read_csv(classifier_csv)
    kmeans_df = pd.read_csv(kmeans_csv)
    merged_df = pd.merge(classifier_df, kmeans_df, on="data_sizes")
    merged_df.to_csv("merged_df.csv")
    return merged_df 